In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras_tuner.tuners import RandomSearch
import keras_tuner

In [2]:
from dataset import load_dataset, load_ica_dataset, load_labels, convert_to_epochs
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features

In [3]:
num_classes = 2
channels = 32
sfreq = 128

In [4]:
#Filtered from SAM40
dataset_ = load_dataset(raw = False)
dataset = convert_to_epochs(dataset_, channels, sfreq)

#ICA filtered
dataset_ica_ = load_ica_dataset()
dataset_ica = convert_to_epochs(dataset_ica_, channels, sfreq)

In [5]:
features = time_series_features(dataset, channels)
features_ica = time_series_features(dataset_ica, channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features
data_ica = features_ica

(120, 25, 32, 128)
(120, 25, 32, 128)


In [6]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

# Linear Regression

In [7]:
def LinearRegression(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
    y_pred = lr_clf.predict(x_test)
    y_true = y_test

    scores_lr = lr_clf.score(x_test, y_test)
    precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
    recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_lr)
    print('precision is:', precision_lr)
    print('recall is:', recall_lr)
    print('f1_score is:', f1_score_lr)
    metrics.confusion_matrix(y_true, y_pred)

# KNN Classifier

In [8]:
def KNN(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    neigh_clf = KNeighborsClassifier(n_neighbors=2)
    neigh_clf.fit(x_train, y_train)
    y_pred = neigh_clf.predict(x_test)
    y_true = y_test

    scores_neigh = neigh_clf.score(x_test, y_test)
    precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
    recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_neigh)
    print('precision is:', precision_neigh)
    print('recall is:', recall_neigh)
    print('f1_score is:', f1_score_neigh)
    metrics.confusion_matrix(y_true, y_pred)

# SVM Classifier

In [9]:
def SVM(data, label):
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    svm_clf = SVC(C=10, kernel='rbf')
    svm_clf.fit(x_train, y_train)
    y_pred = svm_clf.predict(x_test)
    y_pred_train = svm_clf.predict(x_train)
    y_true = y_test

    scores_svm_train = svm_clf.score(x_train, y_train)
    precision_svm_train = metrics.precision_score(y_train, y_pred_train, average='macro')
    recall_svm_train = metrics.recall_score(y_train, y_pred_train, average='micro')
    f1_score_svm_train = metrics.f1_score(y_train, y_pred_train, average='weighted')
    print('accuracy is:', scores_svm_train)
    print('precision is:', precision_svm_train)
    print('recall is:', precision_svm_train)
    print('f1_score is:', f1_score_svm_train)
    metrics.confusion_matrix(y_true, y_pred)


In [10]:
print('Filtered data')
LinearRegression(data,label)
print('\nICA filtered data')
LinearRegression(data_ica,label)

Filtered data
accuracy is: 0.6818181818181818
precision is: 0.6807722500245315
recall is: 0.6818181818181818
f1_score is: 0.6820079922226571

ICA filtered data
accuracy is: 0.5898989898989899
precision is: 0.589167799870141
recall is: 0.5898989898989899
f1_score is: 0.5902679350747364


In [11]:
print('Filtered data')
KNN(data,label)
print('\nICA filtered data')
KNN(data_ica,label)

Filtered data
accuracy is: 0.6343434343434343
precision is: 0.6545246070463582
recall is: 0.6343434343434343
f1_score is: 0.6119252044252045

ICA filtered data
accuracy is: 0.5727272727272728
precision is: 0.5758361534742671
recall is: 0.5727272727272728
f1_score is: 0.5433042081000828


In [12]:
print('Filtered data')
SVM(data,label)
print('\nICA filtered data')
SVM(data_ica,label)

Filtered data
accuracy is: 0.8169154228855722
precision is: 0.8165542710340399
recall is: 0.8165542710340399
f1_score is: 0.8169499129955898

ICA filtered data
accuracy is: 0.7233830845771144
precision is: 0.7264640599969714
recall is: 0.7264640599969714
f1_score is: 0.7230822773350355


# Neural Network

In [13]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data_ica, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

In [14]:
def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model

In [15]:
tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [17]:
tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |?                 |layers
224               |?                 |units_0
relu              |?                 |act_0
896               |?                 |units_1
sigmoid           |?                 |act_1
0.0001            |?                 |learning_rate

Epoch 1/50
57/57 [==============================] - 2s 13ms/step - loss: 0.6896 - accuracy: 0.5372 - val_loss: 0.6749 - val_accuracy: 0.5550
Epoch 2/50
57/57 [==============================] - 0s 7ms/step - loss: 0.6645 - accuracy: 0.5956 - val_loss: 0.6653 - val_accuracy: 0.5917
Epoch 3/50
57/57 [==============================] - 0s 8ms/step - loss: 0.6460 - accuracy: 0.6294 - val_loss: 0.6540 - val_accuracy: 0.6150
Epoch 4/50
57/57 [==============================] - 0s 9ms/step - loss: 0.6285 - accuracy: 0.6683 - val_loss: 0.6394 - val_accuracy: 0.6350
Epoch 5/50
57/57 [==============================] - 0s 7ms/step - loss: 0.6132 - 

FailedPreconditionError: Failed to rename: .\untitled_project\trial_0\checkpoint_temp/part-00000-of-00001.data-00000-of-00001 to: .\untitled_project\trial_0\checkpoint.data-00000-of-00001 : Prosessen får ikke tilgang til filen fordi den brukes av en annen prosess.
; Broken pipe [Op:MergeV2Checkpoints]

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

In [ ]:
scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)